In [10]:

import pandas as pd

# Dataset
df = pd.read_csv(r"C:\Users\The Dev Reda\Desktop\JadAerospace\AircraftParts\Data\Aircraft_Incident_Dataset.csv")
ds = pd.read_csv(r"C:\Users\The Dev Reda\Desktop\JadAerospace\AircraftParts\Data\\jadaero_parts_cleaned.csv")

ds = ds.copy()

ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PartNumber   500 non-null    object
 1   PartName     500 non-null    object
 2   Description  495 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [11]:
IC = df['Incident_Cause(es)'].unique().tolist()
IC

# IMPORTING CATEGORIES AND GIVE IT TO DEEPSEEK TO CREATE A DICTIONARY

['Airplane - Engines, Airplane - Engines - Prop/turbine blade separation, Collision - Object, Collision - Object - Bird, Result - Emergency, forced landing - On runway',
 'Airplane - Undercarriage, Airplane - Undercarriage - Landing gear collapse, Result - Runway mishap',
 'Cargo - Fire/smoke, Result - Damaged on the ground',
 'Result - Damaged on the ground',
 'Landing/takeoff - Tailstrike, Result - Runway mishap',
 'Result - Runway excursion',
 'Info-Unavailable',
 'Result - Runway mishap',
 'Result - Loss of control',
 'Result - Loss of control, Result - Loss of control (presumed)',
 'Landing/takeoff - Landing, Landing/takeoff - Landing - Heavy, Result - Runway mishap',
 'Result - Emergency, forced landing - Ditching',
 'Landing/takeoff - Takeoff - Aborted, Result - Runway excursion',
 'Landing/takeoff - Landing, Landing/takeoff - Landing - Wrong runway/taxiway, Result - Runway excursion',
 'Airplane - Engines, Airplane - Engines - All engine powerloss, Result - Emergency, forced la

In [12]:
# CREATED BY DEEPSEEK

damage_patterns = {
    # Landing Gear Systems
    "landing_gear": {
        "keywords": [
            "undercarriage", "landing gear", "gear collapse", "gear-up landing",
            "tire failure", "brake failure", "strut failure", "retraction failure",
            "oleo failure", "wheel assembly"
        ],
        "common_parts": [
            "MLG-", "NLG-", "STRUT-", "ACTUATOR-GEAR", "SHOCK-ABSORB", 
            "BRAKE-ASSY", "WHEEL-", "TIRE-", "HYDRAULIC-CYL", "TORQUE-LINK"
        ],
        "failure_modes": [
            "Structural collapse", "Hydraulic leakage", "Bearing seizure",
            "Tire blowout", "Brake overheating", "Shimmy vibration"
        ]
    },
    
    # Engine Components
    "engines": {
        "keywords": [
            "engine failure", "powerloss", "blade separation", "fuel starvation",
            "uncontained failure", "compressor stall", "turbine damage",
            "FOD damage", "oil contamination", "fuel pump failure"
        ],
        "common_parts": [
            "HPT-", "LPT-", "HPC-", "LPC-", "COMB-", "NOZZLE-", "FUEL-", 
            "OIL-", "IGNITER-", "VANE-", "BLADE-", "SEAL-", "BEARING-"
        ],
        "failure_modes": [
            "Metal fatigue", "Foreign object damage", "Hot corrosion",
            "Oil starvation", "Combustion instability", "Vibration-induced cracking"
        ]
    },
    
    # Flight Control Systems
    "flight_controls": {
        "keywords": [
            "control surface", "elevator", "rudder", "aileron", "flap", "slat",
            "stab trim", "actuator failure", "cable break", "servo malfunction"
        ],
        "common_parts": [
            "ACTUATOR-", "SERVO-", "CABLE-", "PUSHROD-", "BELLCRANK-", 
            "HORN-", "TAB-", "HINGE-", "TORQUE-TUBE", "JACKSCREW"
        ],
        "failure_modes": [
            "Jamming", "Freeplay", "Corrosion", "Lubrication failure",
            "Electrical fault", "Hydraulic leakage"
        ]
    },
    
    # Avionics/Electrical
    "avionics": {
        "keywords": [
            "transceiver", "weather radar", "altimeter", "navigation failure",
            "electrical fire", "short circuit", "instrument failure",
            "autopilot", "pitot", "static port"
        ],
        "common_parts": [
            "XMTR-", "RCVR-", "ALT-", "NAV-", "ADC-", "FMC-", "BUS-",
            "BREAKER-", "WIRE-", "CONNECTOR-", "SENSOR-", "PROBE-"
        ],
        "failure_modes": [
            "EM interference", "Moisture ingress", "Connector corrosion",
            "Software glitch", "Power surge", "Ground fault"
        ]
    },
    
    # Structural Elements
    "airframe": {
        "keywords": [
            "structural failure", "crack", "corrosion", "fatigue", "spar",
            "rib", "skin", "bulkhead", "frame", "tailstrike"
        ],
        "common_parts": [
            "SPAR-", "RIB-", "SKIN-", "FRAME-", "BULKHEAD-", "DOOR-",
            "PANEL-", "FAIRING-", "SUPPORT-", "BRACKET-"
        ],
        "failure_modes": [
            "Stress corrosion", "Fatigue cracking", "Impact damage",
            "Improper repair", "Galvanic corrosion", "Overstress"
        ]
    },
    
    # Fuel Systems
    "fuel_systems": {
        "keywords": [
            "fuel leak", "fuel contamination", "fuel pump", "fuel valve",
            "fuel filter", "fuel tank", "fuel line"
        ],
        "common_parts": [
            "PUMP-", "VALVE-", "FILTER-", "TANK-", "LINE-", "COUPLING-",
            "VENT-", "PROBE-", "GAUGE-", "SEAL-"
        ],
        "failure_modes": [
            "Seal degradation", "Microbial growth", "Vapor lock",
            "Electrical arcing", "Pressure fluctuation"
        ]
    }
}

# Match part numbers and descriptions to categories
def categorize_part(part_number, description):
    description = str(description).lower()
    for category, data in damage_patterns.items():
        # Check if part number matches known patterns
        if any(common_parts in part_number for common_parts in data['common_parts']):
            return category
        # Check description keywords
        if any(keyword in description for keyword in data['keywords']):
            return category
    return 'other'

# Apply to your DataFrame
ds['damage_category'] = ds.apply(lambda x: categorize_part(x['PartNumber'], x['Description']), axis=1)

ds[ds['damage_category'] != 'other'].sample(10)

,PartNumber,PartName,Description,damage_category
359,2041446-0401,ANTENNA-WEATHER RADAR,Part Number: 2041446-0401\r\rDescription: ANTE...,avionics
357,65-73762-21,NLG ASSY,Part Number: 65-73762-21\r\rDescription: NOSE ...,landing_gear
231,930-2000-010,XMTR RCVR-WEATHER RADAR,Part Number: 930-2000-010\r\rDescription: TRAN...,avionics
383,0856TT1,"TUBE, PITOT",Part Number: 0856TT1\r\rDescription: PITOT STA...,avionics
211,PH1100-MU-L-1,PITOT TUBE,Part Number: PH1100-MU-L-1\r\rDescription: PIT...,avionics
405,2061-15-1,POSITION INDICATOR,Part Number: 2061-15-1\r\rDescription: FLAP PO...,flight_controls
220,FE205-000,RUDDER DAMPE,Part Number: FE205-000\r\rDescription: RUDDER ...,flight_controls
279,0851CM-2,PITOT PROBE,Part Number: 0851CM-2\r\rDescription: PITOT TU...,avionics
182,822-0334-003,TRANSCEIVER,Part Number: 822-0334-003\r\rDescription: TRAN...,avionics
110,9000408-22,TRANSMITTER,Part Number: 9000408-22\r\rDescription: FLAP P...,flight_controls


In [13]:
ds = ds.rename(columns={'damage_category': 'DamageCategory'})
ds[ds['DamageCategory'] != 'other'].sample(10)

#RENAME COLUMN DamageCategory

,PartNumber,PartName,Description,DamageCategory
357,65-73762-21,NLG ASSY,Part Number: 65-73762-21\r\rDescription: NOSE ...,landing_gear
373,GM124-1402-3,SKIN ASSY,Part Number: GM124-1402-3\r\rDescription: SKIN...,airframe
383,0856TT1,"TUBE, PITOT",Part Number: 0856TT1\r\rDescription: PITOT STA...,avionics
307,066-50008-0407,RADAR TRANSCEIVER,Part Number: 066-50008-0407\r\rDescription: TR...,avionics
112,327000-102M01,FTU-RUDDER PEDAL,Part Number: 327000-102M01\r\rDescription: UNI...,flight_controls
445,GM148-1003-2,FLAP ASSY,Part Number: GM148-1003-2\r\rDescription: FLAP...,flight_controls
202,0856AE19,PITOT TUBE,Part Number: 0856AE19\r\rDescription: PITOT ST...,avionics
170,568-1-27202-05R,FUEL PUMP,Part Number: 568-1-27202-05R\r\rDescription: F...,fuel_systems
405,2061-15-1,POSITION INDICATOR,Part Number: 2061-15-1\r\rDescription: FLAP PO...,flight_controls
105,622-5132-633,RADAR TRANSCEIVER,Part Number: 622-5132-633\r\rDescription: WEAT...,avionics


In [14]:
ds['DamageCategory'].value_counts()

# SHOW THE UNIQUE VALUES IN THE DamageCategory COLUMN
# 27 ROWS FROM 500 ROWS

DamageCategory
other              473
avionics            15
flight_controls      9
fuel_systems         1
landing_gear         1
airframe             1
Name: count, dtype: int64

In [15]:
ds = ds[ds['DamageCategory'] != 'other']
ds.sample(10)

,PartNumber,PartName,Description,DamageCategory
112,327000-102M01,FTU-RUDDER PEDAL,Part Number: 327000-102M01\r\rDescription: UNI...,flight_controls
405,2061-15-1,POSITION INDICATOR,Part Number: 2061-15-1\r\rDescription: FLAP PO...,flight_controls
469,622-7292-101,TRANSCEIVER,Part Number: 622-7292-101\r\rDescription: TRAN...,avionics
110,9000408-22,TRANSMITTER,Part Number: 9000408-22\r\rDescription: FLAP P...,flight_controls
383,0856TT1,"TUBE, PITOT",Part Number: 0856TT1\r\rDescription: PITOT STA...,avionics
307,066-50008-0407,RADAR TRANSCEIVER,Part Number: 066-50008-0407\r\rDescription: TR...,avionics
183,822-2764-050,TRANSCEIVER,Part Number: 822-2764-050\r\rDescription: HFS ...,avionics
373,GM124-1402-3,SKIN ASSY,Part Number: GM124-1402-3\r\rDescription: SKIN...,airframe
211,PH1100-MU-L-1,PITOT TUBE,Part Number: PH1100-MU-L-1\r\rDescription: PIT...,avionics
279,0851CM-2,PITOT PROBE,Part Number: 0851CM-2\r\rDescription: PITOT TU...,avionics
